In [ ]:
import sys
import logging
import argparse
import itertools

import numpy as np
from joblib import Parallel, delayed

sys.path.insert(0, 'network')
from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse
from sequences import GaussianSequence

logging.basicConfig(level=logging.INFO)

def main(args):

    sigma_z = [0, 1, 2.5, 10]
    S = [1, 4]
    P = [16]

    def func(sigma_z, S, P):
        M, overlaps = simulate_network(sigma_z, S, P)
        return ((sigma_z, S, P), M, overlaps)

    data = Parallel(n_jobs=args.cpu_cores)(delayed(func)(*args) for args in itertools.product(sigma_z, S, P))

    if args.save:
        logging.info("Saving data")
        np.save(open(args.data_path, "wb"), data)

def simulate_network(z_pert, S=1, P=16):

    #phi = ErrorFunction(mu=0.22, sigma=0.1).phi
    phi = ErrorFunction(mu=0.14, sigma=0.1).phi
    exc = Population(N=40000, tau=1e-2, phi=phi)

    sequences = [GaussianSequence(P,exc.size,seed=i) for i in range(S)]
    patterns = np.stack([s.inputs for s in sequences])

    conn_EE = SparseConnectivity(source=exc, target=exc, p=0.005, seed=42)
    synapse = LinearSynapse(conn_EE.K, A=1)
    conn_EE.store_sequences(patterns, synapse.h_EE)

    net = RateNetwork(exc, c_EE=conn_EE, formulation=1)

    z0 = z_pert*np.random.RandomState(seed=43).normal(0,1,size=exc.size)
    r0 = exc.phi(patterns[0,0,:] + z0)
    net.simulate(0.35, r0=r0)

    M = np.mean(net.exc.state**2, axis=0)
    overlaps = sequences[0].overlaps(net, exc)

    return M, overlaps


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('-s', '--save', default=True)
    parser.add_argument('-c', '--cpu_cores', default=8)
    parser.add_argument('-d', '--data_path', default='figures/supplement/11/data/data_b_region_e.npy')
    args = parser.parse_args()

    main(args) 